In [1]:
import numpy as np

class CrossValidator:
    def __init__(self, model, X, y, k=5, shuffle=True, random_seed=None):
        self.model = model
        self.X = np.array(X)
        self.y = np.array(y)
        self.k = k
        self.shuffle = shuffle
        self.random_seed = random_seed

    def split(self):
        n_samples = len(self.X)
        indices = np.arange(n_samples)

        if self.shuffle:
            rng = np.random.default_rng(self.random_seed)
            rng.shuffle(indices)

        fold_sizes = np.full(self.k, n_samples // self.k, dtype=int)
        fold_sizes[:n_samples % self.k] += 1

        current = 0
        folds = []
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_idx = indices[start:stop]
            train_idx = np.concatenate([indices[:start], indices[stop:]])
            folds.append((train_idx, test_idx))
            current = stop
        return folds

    def evaluate(self, scoring_func):
        scores = []
        for train_idx, test_idx in self.split():
            X_train, y_train = self.X[train_idx], self.y[train_idx]
            X_test, y_test = self.X[test_idx], self.y[test_idx]

            self.model.fit(X_train, y_train)
            y_pred = self.model.predict(X_test)
            score = scoring_func(y_test, y_pred)
            scores.append(score)
        return scores
